# Building RAG-Enhanced LLM for Travel Guide Conversational Application

In this code file, I will build the conversational application from scratch. In other words, I will increase the features added to the conversational application. At the beginning, the vanilla application is based on the parametric knowledge of the LLM.

In [1]:
# Setting up the environment through OpenAI's API
import os
from dotenv import load_dotenv
#from langchain import HuggingFaceHub

from langchain import PromptTemplate, LLMChain

load_dotenv()

#os.environ["HUGGINGFACEHUB_API_TOKEN"]
#retrieving api key
key=os.environ['OPENAI_API_KEY']

## Conversational Application Without RAG and Memory

In [2]:
# Define and manage prompts for the LLM
# Especially this helps the output to be more conversational. Without this, the output will be concise and rigid, even robot-like
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# Represent the messages in a structured format
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

# Build and execute chains of language models
from langchain.chains import LLMChain, ConversationChain  # passes input to an LLM and returns the output; manage and maintain conversational history
from langchain.chat_models import ChatOpenAI  # Choose OpenAI's LLM (default to GPT-3.5-turbo)

chat = ChatOpenAI()
# If I want to use another version of GPT, the code is
# chat = ChatOpenAI(model_name="gpt-4-turbo")

# SystemMessage defines the tone of the assistant. You can make the assistant helpful or angry with just the SystemMessage
# HumanMessage can be any question you want to ask the assistant
messages = [
    SystemMessage(content="You are a helpful and attentive assistant that help the user to plan an optimized itinerary."),
    HumanMessage(content="I will have one free day in Kyoto. What should I do to best enjoy myself?")
]
output = chat(messages)
print(output.content)

/tmp/ipykernel_1412/2738840191.py:21: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI()
/tmp/ipykernel_1412/2738840191.py:31: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output = chat(messages)


To make the most of your free day in Kyoto and ensure you have a great time, I recommend the following itinerary:

1. **Arashiyama Bamboo Grove**: Start your day early by visiting the beautiful Arashiyama Bamboo Grove. Take a leisurely stroll through the serene bamboo forest and enjoy the peaceful atmosphere.

2. **Tenryu-ji Temple**: After exploring the bamboo grove, visit Tenryu-ji Temple, which is located nearby. This temple is known for its stunning garden and picturesque views of the Arashiyama mountains.

3. **Kinkaku-ji (Golden Pavilion)**: Head to Kinkaku-ji, also known as the Golden Pavilion, one of Kyoto's most iconic landmarks. Marvel at the breathtaking golden temple set against a backdrop of lush greenery and a tranquil pond.

4. **Lunch**: Grab a delicious and authentic Japanese lunch at a local restaurant in the area. Don't miss the opportunity to try some Kyoto specialties such as Kyo kaiseki, tofu dishes, or matcha sweets.

5. **Fushimi Inari Taisha**: In the afternoon

### Observation
With just one sentence contain the time I can have in Kyoto (1 day) and one question asking for advice, the conversational application can generate a whole itinery from morning until evening, which includes very famous Kyoto destinations (such as Kiyomizu-dera). However, often we do not end a conversation with just 1 turn of interaction. Normally, as we have more information and we start to link the information with our wants, needs, and curiosity, we will ask more questions based on the previous interactions and context in the conversation. In this case, we need the bot to have a memory so that we can optimize the travel plans based on our likings and conditions and the bot's previous suggestions. 

## Conversational Application With Memory

Note that the bot will only start to have a memory after including these specific code (in the next box). The conversation I had in the previous section will not be included in the memory of the conversational application created in this section.

In [ ]:
# ConversationBufferMemory stores the entire conversation history (both user inputs and AI responses) in a buffer
# It allows the chatbot to maintain context across multiple exchanges.
from langchain.memory import ConversationBufferMemory

# ConversationChain connects a language model (LLM) to a memory component, 
# enabling contextual conversations over multiple turns
from langchain.chains import ConversationChain  

# verbose=True for easier debugging if needed, prints detailed logs of inputs and outputs
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=chat, verbose=True, memory=memory
)

# Run a conversation with run. You need to call this every time you want to interact with the bot
# This helps with: missing context in memory, incorrect formatting of the user input, unintended behavior due to chain logic
conversation.run("What should I do in Kyoto if I just have one day?")

/tmp/ipykernel_1383/3004699516.py:10: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
/tmp/ipykernel_1383/3004699516.py:11: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(
/tmp/ipykernel_1383/3004699516.py:17: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  conversation.run("What should I do in Kyoto if I just have one day?")




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What should I do in Kyoto if I just have one day?
AI:

> Finished chain.


"There are so many amazing things to do in Kyoto in just one day! I would recommend starting your day by visiting the iconic Fushimi Inari Shrine with its thousands of red torii gates. Then, you can head to Kiyomizu-dera Temple for stunning views of the city. Don't forget to try some traditional Kyoto cuisine like matcha green tea and kaiseki ryori for lunch. In the afternoon, you can explore the historic Gion district and maybe even catch a glimpse of a geisha. Finally, end your day with a relaxing stroll through the beautiful Arashiyama Bamboo Grove. Enjoy your day in Kyoto!"

In [5]:
conversation.run("I visited Kyomizu-dera before. I want to focus my day for the best cuisine in Kyoto. Give me some suggestions.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What should I do in Kyoto if I just have one day?
AI: There are so many amazing things to do in Kyoto in just one day! I would recommend starting your day by visiting the iconic Fushimi Inari Shrine with its thousands of red torii gates. Then, you can head to Kiyomizu-dera Temple for stunning views of the city. Don't forget to try some traditional Kyoto cuisine like matcha green tea and kaiseki ryori for lunch. In the afternoon, you can explore the historic Gion district and maybe even catch a glimpse of a geisha. Finally, end your day with a relaxing stroll through the beautiful Arashiyama Bamboo Grove. Enjoy your day in Kyoto!
Human: I visited Kyomizu-dera bef

"If you're looking to focus on the best cuisine in Kyoto, I would recommend trying out some of the following options:\n\n1. Kaiseki Ryori: This traditional multi-course meal is a must-try in Kyoto. It typically features seasonal and local ingredients prepared in a beautifully presented manner.\n\n2. Yudofu: A simple yet delicious dish made with tofu simmered in a hot pot. It's a popular choice for vegetarians and those looking for a light and healthy meal.\n\n3. Sushi: While Kyoto is not known for its sushi like other cities in Japan, you can still find some excellent sushi restaurants that use fresh, local ingredients.\n\n4. Yuba: A specialty of Kyoto, yuba is made from the skin that forms on top of heated soy milk. It can be enjoyed in various dishes such as yuba sushi or yuba hot pot.\n\n5. Matcha Green Tea: Kyoto is famous for its matcha green tea, so be sure to try a matcha-flavored dessert like matcha ice cream or matcha parfait.\n\nI hope these suggestions help you have a memora

### Observation:
Based on the output, the conversational application has a memory now, and I have actually engaged in a conversation with the bot. However, there are two things which make the above code less suitable for real-life application. Firstly, the bot's behavior is rigid ("The following is a friendly conversation between a human and an AI..."). In a real-life scenario, we want to customize the bot's behavior, such as being attentive, cheerful, or thoughtful. Secondly, users often interact through a GUI, not through code like conversation.run("input_question"). In the following code, I will address these issues

In [3]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.memory import ConversationBufferMemory

# Dynamically construct the prompt that the model will see.
# Customize the model's behavior (e.g. helpful, cheerful, or angry)
prompt = ChatPromptTemplate.from_messages(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a helpful, cheerful, and attentive assistant that help the user to plan an optimized itinerary."
        ),
        # The `variable_name` here is what must align with memory
        MessagesPlaceholder(variable_name="chat_history"),  # Dynamically inject the memory's conversation history into the prompt
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

# Notice that we `return_messages=True` to fit into the MessagesPlaceholder
# Notice that `"chat_history"` aligns with the MessagesPlaceholder name
# memory_key="chat_history" links the memory to the MessagesPlaceholder, so the previous exchanges are injected into the prompt.
# return_messages=True ensures that memory stores full messages (not just text) in a structured format, 
# which is necessary for the ChatPromptTemplate to include formatted message objects.
memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True)
conversation = LLMChain(
    llm=chat,
    prompt=prompt,
    verbose=False,
    memory=memory
)

/tmp/ipykernel_1412/2173986566.py:27: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history",return_messages=True)
/tmp/ipykernel_1412/2173986566.py:28: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  conversation = LLMChain(


In [4]:
while True:
    query = input('You: ')
    if query == 'quit':
        break
    output = conversation({"question": query})
    print('User: ', query)
    print('AI system: ', output['text'])

/tmp/ipykernel_1412/2871464895.py:5: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output = conversation({"question": query})


User:  Give me some must-see places for anime fans
AI system:  Of course! If you're an anime fan, here are some must-see places that you should consider including in your itinerary:

1. Akihabara, Tokyo, Japan - Known as the mecca for anime and manga fans, Akihabara is a bustling district filled with anime shops, maid cafes, and electronic stores.

2. Studio Ghibli Museum, Tokyo, Japan - This museum is dedicated to the renowned animation studio, Studio Ghibli, and showcases the work of Hayao Miyazaki and other talented artists.

3. Nakano Broadway, Tokyo, Japan - This shopping complex is a paradise for anime and manga enthusiasts, with numerous shops selling a wide variety of collectibles, merchandise, and rare items.

4. Kyoto International Manga Museum, Kyoto, Japan - This museum houses a vast collection of manga from different eras and genres, making it a great place to learn about the history of Japanese comics.

5. Pokémon Center, various locations in Japan - For Pokémon fans, vis

In [8]:
# (Only self-learning here)
# The keys in output:
# for key in output.keys():
#     print(key)

question
chat_history
text


### Further self-learning
Since the contents of the chat might be useful in the future, I will write code to save the whole chat history after quitting. 

In [ ]:
# Bonus step: Save the conversation history after quitting
# folder_path = "/mnt/c/Users/haanh/rag-travel-guide/RAG-Enhanced-Travel-Guide-Bot/prev_convo"

# # Find the next available file number
# existing_files = [f for f in os.listdir(folder_path) if f.startswith("conversation_history_") and f.endswith(".txt")]
# file_numbers = [int(f.split("_")[-1].split(".")[0]) for f in existing_files if f.split("_")[-1].split(".")[0].isdigit()]
# next_file_number = max(file_numbers) + 1 if file_numbers else 1  # Start from 1 if no files exist

# # Define the file path with the new number
# file_path = os.path.join(folder_path, f"conversation_history_{next_file_number}.txt")

# # Access the conversation history
# history = memory.buffer

# # Write to a numbered text file
# with open(file_path, 'w') as file:
#     for message in history:
#         if isinstance(message, HumanMessage):
#             file.write(f'User: {message.content}\n')
#         elif isinstance(message, AIMessage):
#             file.write(f'AI system: {message.content}\n')

In other cases, I might want to use the chat history of a previous conversation as the context of a new chat. The following code allows me to do that:

In [ ]:
# from langchain.prompts import (
#     ChatPromptTemplate,
#     MessagesPlaceholder,
#     SystemMessagePromptTemplate,
#     HumanMessagePromptTemplate,
# )
# from langchain.memory import ConversationBufferMemory

# # Step 1: Define the prompt
# prompt = ChatPromptTemplate.from_messages(
#     messages=[
#         SystemMessagePromptTemplate.from_template(
#             "You are a helpful, cheerful, and attentive assistant that helps the user to plan an optimized itinerary."
#         ),
#         MessagesPlaceholder(variable_name="chat_history"),  # Dynamically inject memory's conversation history
#         HumanMessagePromptTemplate.from_template("{question}")
#     ]
# )

# # Step 2: Load the saved chat history from the file
# def load_chat_history(file_path):
#     chat_history = []
#     with open(file_path, 'r') as file:
#         for line in file:
#             if line.startswith('User:'):
#                 chat_history.append(HumanMessage(content=line.strip().replace('User: ', '')))
#             elif line.startswith('AI system:'):
#                 chat_history.append(AIMessage(content=line.strip().replace('AI system: ', '')))
#     return chat_history

# # Load the chat history from the saved file
# file_path = '/mnt/c/Users/haanh/rag-travel-guide/RAG-Enhanced-Travel-Guide-Bot/prev_convo/conversation_history_1.txt'
# previous_chat_history = load_chat_history(file_path)

# # Step 3: Create a new memory instance
# new_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# # Step 4: Load the previous chat history into the new memory
# for message in previous_chat_history:
#     new_memory.save_context(message)

# # Step 5: Create a new conversation with the loaded memory
# new_conversation = LLMChain(
#     llm=chat,
#     prompt=prompt,
#     verbose=False,
#     memory=new_memory
# )

# # Start a new conversation
# while True:
#     query = input('You: ')
#     if query == 'quit':
#         break
#     output = new_conversation({"question": query})
#     print('User: ', query)
#     print('AI system: ', output['text'])


In some cases, I want to include multiple chat history files as the context of the new chat. The following code allows me to do this:

In [ ]:
# from langchain.prompts import (
#     ChatPromptTemplate,
#     MessagesPlaceholder,
#     SystemMessagePromptTemplate,
#     HumanMessagePromptTemplate,
# )
# from langchain.memory import ConversationBufferMemory

# import glob

# # Step 1: Define the prompt
# prompt = ChatPromptTemplate.from_messages(
#     messages=[
#         SystemMessagePromptTemplate.from_template(
#             "You are a helpful, cheerful, and attentive assistant that helps the user to plan an optimized itinerary."
#         ),
#         MessagesPlaceholder(variable_name="chat_history"),  # Dynamically inject memory's conversation history
#         HumanMessagePromptTemplate.from_template("{question}")
#     ]
# )

# # Step 2: Load chat history from multiple files
# def load_chat_history(file_paths):
#     chat_history = []
#     for file_path in file_paths:
#         with open(file_path, 'r') as file:
#             for line in file:
#                 if line.startswith('User:'):
#                     chat_history.append(HumanMessage(content=line.strip().replace('User: ', '')))
#                 elif line.startswith('AI system:'):
#                     chat_history.append(AIMessage(content=line.strip().replace('AI system: ', '')))
#     return chat_history

# # Step 3: Specify the directory and file pattern for loading chat history
# directory_path = '/mnt/c/Users/haanh/rag-travel-guide/RAG-Enhanced-Travel-Guide-Bot/prev_convo'  
# file_pattern = '*.txt'
# file_paths = glob.glob(f"{directory_path}/{file_pattern}")

# # Load the chat history from the specified files
# previous_chat_history = load_chat_history(file_paths)

# # Step 4: Create a new memory instance
# new_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# # Step 5: Load the previous chat history into the new memory
# for message in previous_chat_history:
#     new_memory.save_context(message)

# # Step 6: Create a new conversation with the loaded memory
# new_conversation = LLMChain(
#     llm=chat,
#     prompt=prompt,
#     verbose=False,
#     memory=new_memory
# )

# # Start a new conversation
# while True:
#     query = input('You: ')
#     if query == 'quit':
#         break
#     output = new_conversation({"question": query})
#     print('User: ', query)
#     print('AI system: ', output['text'])


## Conversational Application with Memory and Document Retrieval Component

Here, I want my chatbot to have extra knowledge about Japan, which comes from a personal document. This document was written by the Japan National Tourism Organization. 

I will implement a vector-store-backed retriever utilizing the ConversationalRetrievalChain. This chain employs a retriever to access the given knowledge base while incorporating chat history, which can be provided as a parameter using the selected memory type

In [6]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)

raw_document = PyPDFLoader('/mnt/c/Users/haanh/rag-travel-guide/RAG-Enhanced-Travel-Guide-Bot/data/japan-where-luxury-comes-to-life.pdf').load()
document = text_splitter.split_documents(raw_document)
db = FAISS.from_documents(document, OpenAIEmbeddings())
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
llm = ChatOpenAI()

/tmp/ipykernel_1412/3970310402.py:14: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  db = FAISS.from_documents(document, OpenAIEmbeddings())


In [9]:
# Check if the LLM can understand the external document
qa_chain = ConversationalRetrievalChain.from_llm(llm, retriever=db.as_retriever(), memory=memory, verbose=True)
qa_chain.run({'question':'Tell me about Okada Museum of Art'})



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Tell me about Okada Museum of Arts
Assistant: The Okada Museum of Art is a museum located in Japan. Unfortunately, there are no specific details or information provided in the context given about the Okada Museum of Art.
Follow Up Input: Tell me about Okada Museum of Art
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
ADACHI MUSEUM OF ART  
BIZEN OSAFUNE JAPANESE SWORD MUSEUM  
EXPLORE NISHI-AWA REGION  
GINZAN ONSEN  
HANBE GARDEN  
HISTORICAL SOMARO MAIKO TEAHOUSE  
HOSOO TEXTILES 

"The Okada Museum of Art is located in Japan and is known for its impressive collection of art. Unfortunately, I don't have specific details about the museum's offerings or location."

In [10]:
# Check if the LLM can understand the external document
qa_chain = ConversationalRetrievalChain.from_llm(llm, retriever=db.as_retriever(), memory=memory, verbose=True)
qa_chain.run({'question':'Tell me about Asaba Ryokan'})



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Tell me about Okada Museum of Arts
Assistant: The Okada Museum of Art is a museum located in Japan. Unfortunately, there are no specific details or information provided in the context given about the Okada Museum of Art.
Human: Tell me about Okada Museum of Art
Assistant: The Okada Museum of Art is located in Japan and is known for its impressive collection of art. Unfortunately, I don't have specific details about the museum's offerings or location.
Follow Up Input: Tell me about Asaba Ryokan
Standalone question:

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you do

"I don't have information specifically about Asaba Ryokan from the provided context."

### Observation:
Even though the PDF has information about Okada Museum of Art and Asaba Ryokan, such as their addresses and what services are offered there, the retriever cannot retrieve any information. This might happen because there are so many pictures in the PDF file, which makes the LLM confused and unable to retrieve the texts in the PDF. 

In the following run, I will retry using a text-only PDF file. 

In [2]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)

raw_document = PyPDFLoader('/mnt/c/Users/haanh/rag-travel-guide/RAG-Enhanced-Travel-Guide-Bot/data/japan_14_day_guide.pdf').load()
document = text_splitter.split_documents(raw_document)
db = FAISS.from_documents(document, OpenAIEmbeddings())
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
llm = ChatOpenAI()

/tmp/ipykernel_1253/1766139186.py:14: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  db = FAISS.from_documents(document, OpenAIEmbeddings())
/tmp/ipykernel_1253/1766139186.py:15: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
/tmp/ipykernel_1253/1766139186.py:16: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:

In [3]:
# Check if the LLM can understand the external document
qa_chain = ConversationalRetrievalChain.from_llm(llm, retriever=db.as_retriever(), memory=memory, verbose=True)
qa_chain.run({'question':'Tell me about Kenrokuen'})

/tmp/ipykernel_1253/4284236744.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa_chain.run({'question':'Tell me about Kenrokuen'})




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Kenrokuen will help with understanding this landscape theory as the grounds are teeming with 
water features, bridges, teahouses, trees, flowers, stones, viewpoints and hidden nooks. 
Water is diverted from a distant river by a sophisticated water system constructed in 1632 to feed 
Kenrokuen's various streams and ponds including the two main ponds in the garden, Kasumigaike 
and Hisagoike. The over two-meter tall Kotojitoro Lantern, uniquely built with two legs instead of 
one, stands at the northern bank of Kasumigaike Pond and is an iconic symbol of Kenrokuen. In

addition, a fountain below Kasumigaike Pond is one of Japan's oldest, and is powered by the drop 
in elevation from the pond which caus

'Kenrokuen is a renowned garden located in Kanazawa, Japan. It is considered one of Japan\'s "three most beautiful gardens" and was originally the outer garden of Kanazawa Castle. Kenrokuen features a variety of flowering trees that give the garden a unique look each season. The name Kenrokuen means "Garden of the six sublimities" which refers to spaciousness, seclusion, artificiality, antiquity, abundant water, and broad views - all key elements in Chinese landscape theory. The garden is filled with water features, bridges, teahouses, trees, flowers, stones, viewpoints, and hidden nooks, making it an ideal place to understand landscape theory and enjoy the beauty of nature.'

In [4]:
# Check if the LLM can understand the external document
qa_chain = ConversationalRetrievalChain.from_llm(llm, retriever=db.as_retriever(), memory=memory, verbose=True)
qa_chain.run({'question':'Advise when it would be a good time to visit Kenrokuen.'})



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Tell me about Kenrokuen
Assistant: Kenrokuen is a famous garden located in Kanazawa, Japan. It is considered one of Japan's "three most beautiful gardens" alongside Mito's Kairakuen and Okayama's Korakuen. The garden used to be the outer garden of Kanazawa Castle and was constructed by the ruling Maeda family over nearly two centuries. Kenrokuen features a variety of flowering trees that provide the garden with a different look every season. The name Kenrokuen means "Garden of the six sublimities," representing spaciousness, seclusion, artificiality, antiquity, abundant water, and broad views, which are essential attributes according to Chinese landscape theory. The garden is teeming with water features, bridges, teahouses, trees, flowers, stones, viewpoints, a

'A good time to visit Kenrokuen would be during the changing seasons to appreciate the different looks the garden offers. In spring, you can see plum blossoms from late February to early April, and cherry blossoms from late March to early April. In autumn, the fall colors of cherry and maple trees are prominent from late November to early December. Each season offers a unique experience at Kenrokuen.'

### Observation:
The RAG-enhanced LLM can now extract correct information from the provided PDF file when the PDF file is purely text-based. Simultaneously, it must be noted that the PDF file here is just a 9-page document. In reality, we want the reitrievl component to read multiple (long) documents. In the next section, I will try adding a long and more complicated document and ask the conversational application again. 

In [4]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)

# List of PDF file paths
pdf_paths = [
    "/mnt/c/Users/haanh/rag-travel-guide/RAG-Enhanced-Travel-Guide-Bot/data/japan_14_day_guide.pdf",
    "/mnt/c/Users/haanh/rag-travel-guide/RAG-Enhanced-Travel-Guide-Bot/data/japan_visa.pdf"
]

# Load and split documents from all PDFs
documents = []
for path in pdf_paths:
    raw_docs = PyPDFLoader(path).load()
    documents.extend(text_splitter.split_documents(raw_docs))

db = FAISS.from_documents(documents, OpenAIEmbeddings())

memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
llm = ChatOpenAI()

In [7]:
# Check if the LLM can understand the external document
qa_chain = ConversationalRetrievalChain.from_llm(llm, retriever=db.as_retriever(), memory=memory, verbose=True)
qa_chain.run({'question':'Give me the actions I need to take as a Vietnamese tourist to Japan, from before my arrival in Japan to where I should visit in Japan.'})



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Give me the actions I need to take as a Vietnamese tourist to Japan, from before my arrival in Japan to what I can do in Japan.
Assistant: As a Vietnamese tourist planning to travel to Japan, here are the steps you need to take:

1. **Before Arrival in Japan:**
   a. Contact the Japanese Embassy or Consulate in Vietnam or a neighboring country to inquire about visa requirements.
   b. Collect the necessary documents for a Japan visa application, including your passport, recent passport-size photos, filled Japan Visa Application Form, booked flight ticket, and proof of financial requirements.
   c. If you intend to apply for a long-term visa, you will need to first apply for a Certificate of Eligibility (COE) from an Immigration Office in Japan through a proxy.


'As a Vietnamese tourist visiting Japan, you do need a visa unless you are exempt. Fortunately, citizens of Vietnam are not currently on the list of countries exempt from visa requirements for short-term stays in Japan. Therefore, you would need to apply for a visa before traveling to Japan.'

### Observation:
The RAG-enhanced LLM can only talk about visa requirements for Vietnamese tourists. It failed to answer the part where I want to know where I should visit in Japan. 

In the next run, I will just ask about tourist attractions in Japan and see what answer the RAG-enhanced LLM would give me.

In [8]:
# Check if the LLM can understand the external document
qa_chain = ConversationalRetrievalChain.from_llm(llm, retriever=db.as_retriever(), memory=memory, verbose=True)
qa_chain.run({'question':'Should I visit Hama Rikyu when I am in Japan?'})



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: Give me the actions I need to take as a Vietnamese tourist to Japan, from before my arrival in Japan to what I can do in Japan.
Assistant: As a Vietnamese tourist planning to travel to Japan, here are the steps you need to take:

1. **Before Arrival in Japan:**
   a. Contact the Japanese Embassy or Consulate in Vietnam or a neighboring country to inquire about visa requirements.
   b. Collect the necessary documents for a Japan visa application, including your passport, recent passport-size photos, filled Japan Visa Application Form, booked flight ticket, and proof of financial requirements.
   c. If you intend to apply for a long-term visa, you will need to first apply for a Certificate of Eligibility (COE) from an Immigration Office in Japan through a proxy.


"If you appreciate traditional Japanese gardens with historical significance and beautiful seasonal views, visiting Hama Rikyu in Japan could be a great addition to your itinerary. The garden offers a peaceful retreat from the urban surroundings and showcases unique features like seawater ponds and a teahouse. It's conveniently located in central Tokyo, making it easily accessible. Whether you decide to visit Hama Rikyu would depend on your interests and the time of year you plan to be there."

### Observation
The RAG-enhanced LLM can answer question about a place mentioned in a provided document in the retriever. I assume that we can only ask concise questions to the RAG-enhanced LLM. If the question is complicated in the sense that if there are sub-questions in our question, the RAG-enhanced LLM is only able to answer one sub-question.

## Combining both external knowledge and LLM's own knowledge
In previous codes, the LLM only used the knowledge from the external documents. Here, I will combined both LLM's external and internal knowledge to answer the question. In other words, I am making the LLM agentic.

In [16]:
from langchain.agents.agent_toolkits import create_retriever_tool
tool = create_retriever_tool(
    db.as_retriever(), 
    "japan_travel",  # name of the retriever tool
    "Searches and returns documents about Japan."  # description of what the retriever tool does
)

tools = [tool]
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature = 0)
agent_executor = create_conversational_retrieval_agent(llm, tools, memory_key='chat_history', verbose=True)

agent_executor({"input": "Give me information about Shirakawa-go."})



> Entering new AgentExecutor chain...

Invoking: `japan_travel` with `{'query': 'Shirakawa-go'}`


Day 13 - Shirakawago to Takayama 
Head to the beautifully preserved old town of Takayama in Gifu Prefecture. 
The Shirakawa-go (白川郷, Shirakawagō) and neighboring Gokayama ( 五箇山) regions line the 
Shogawa River Valley in the remote mountains that span from Gifu to Toyama Prefectures. 
Declared a UNESCO world heritage site in 1995, they are famous for their traditional gassho -
zukuri farmhouses, some of which are more than 250 years old. 
Gassho-zukuri means "constructed like hands in prayer", as the farmhouses' steep thatched roofs 
resemble the hands of Buddhist monks pressed together in prayer. The architectural style 
developed over many generations and is designed to withstand the large amounts of heavy snow 
that falls in the region during winter. The roofs, made without nails, provided a large attic space 
used for cultivating silkworms. 
Ogimachi, Shirakawa -go's largest village 

{'input': 'Give me information about Shirakawa-go.',
 'chat_history': [HumanMessage(content='Give me information about Shirakawa-go.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Shirakawa-go"}', 'name': 'japan_travel'}}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 89, 'total_tokens': 109, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-ab19cd62-bf44-4e15-9e94-fbada4369adc-0'),
  FunctionMessage(content='Day 13 - Shirakawago to Takayama \nHead to the beautifully preserved old town of Takayama in Gifu Prefecture. \nThe Shirakawa-go (白川郷, Shirakawagō) and neighboring Gokayama ( 五箇山) regions line th

In [5]:
from langchain.agents.agent_toolkits import create_retriever_tool
tool = create_retriever_tool(
    db.as_retriever(), 
    "japan_travel",  # name of the retriever tool
    "Searches and returns documents about Japan."  # description of what the retriever tool does
)

tools = [tool]
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature = 0)
agent_executor = create_conversational_retrieval_agent(llm, tools, memory_key='chat_history', verbose=True)

agent_executor({"input": "Give me a detailed description of Shirakawa-go."})

/tmp/ipykernel_1253/1587216568.py:16: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent_executor({"input": "Give me a detailed description of Shirakawa-go."})




> Entering new AgentExecutor chain...

Invoking: `japan_travel` with `{'query': 'Shirakawa-go'}`


Day 13 - Shirakawago to Takayama 
Head to the beautifully preserved old town of Takayama in Gifu Prefecture. 
The Shirakawa-go (白川郷, Shirakawagō) and neighboring Gokayama ( 五箇山) regions line the 
Shogawa River Valley in the remote mountains that span from Gifu to Toyama Prefectures. 
Declared a UNESCO world heritage site in 1995, they are famous for their traditional gassho -
zukuri farmhouses, some of which are more than 250 years old. 
Gassho-zukuri means "constructed like hands in prayer", as the farmhouses' steep thatched roofs 
resemble the hands of Buddhist monks pressed together in prayer. The architectural style 
developed over many generations and is designed to withstand the large amounts of heavy snow 
that falls in the region during winter. The roofs, made without nails, provided a large attic space 
used for cultivating silkworms. 
Ogimachi, Shirakawa -go's largest village 

{'input': 'Give me a detailed description of Shirakawa-go.',
 'chat_history': [HumanMessage(content='Give me a detailed description of Shirakawa-go.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Shirakawa-go"}', 'name': 'japan_travel'}}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 91, 'total_tokens': 111, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-dd5bac07-5a79-46bf-877c-3170eac0c6e3-0'),
  FunctionMessage(content='Day 13 - Shirakawago to Takayama \nHead to the beautifully preserved old town of Takayama in Gifu Prefecture. \nThe Shirakawa-go (白川郷, Shirakawagō) and neighboring Gokayama ( 五箇山)

### Observations:
I asked the conversational application to give me a detailed description of Shirakawa-go but it only gave me a very brief reply. It even included information about a place I did not mention and not relevant, Gokayama.

Now, I will test if the conversational chatbot is using its internal knowledge

In [15]:
from langchain.agents.agent_toolkits import create_retriever_tool
tool = create_retriever_tool(
    db.as_retriever(), 
    "japan_travel",  # name of the retriever tool
    "Searches and returns documents about Japan."  # description of what the retriever tool does
)

tools = [tool]
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

from langchain.agents.agent_toolkits import create_conversational_retrieval_agent
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature = 0)
agent_executor = create_conversational_retrieval_agent(llm, tools, memory_key='chat_history', verbose=True)

agent_executor({"input": "Where should I visit in Scotland?"})



> Entering new AgentExecutor chain...
Scotland is a beautiful country with many amazing places to visit. Some popular destinations in Scotland that you may consider visiting are:

1. Edinburgh - The capital city of Scotland, known for its historic architecture, Edinburgh Castle, Royal Mile, and vibrant cultural scene.

2. Glasgow - Scotland's largest city, famous for its art galleries, museums, shopping, and lively nightlife.

3. The Highlands - Explore the stunning landscapes of the Scottish Highlands, including Loch Ness, Glencoe, and the Isle of Skye.

4. Stirling - Visit Stirling Castle, the Wallace Monument, and enjoy the picturesque views of the surrounding countryside.

5. Inverness - Gateway to the Highlands, Inverness is a charming city with a rich history and close proximity to Loch Ness.

6. Aberdeen - Known as the "Granite City," Aberdeen offers beautiful architecture, sandy beaches, and a bustling harbor.

7. Isle of Arran - A picturesque island off the west coast of Sco

{'input': 'Where should I visit in Scotland?',
 'chat_history': [HumanMessage(content='Where should I visit in Scotland?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Scotland is a beautiful country with many amazing places to visit. Some popular destinations in Scotland that you may consider visiting are:\n\n1. Edinburgh - The capital city of Scotland, known for its historic architecture, Edinburgh Castle, Royal Mile, and vibrant cultural scene.\n\n2. Glasgow - Scotland\'s largest city, famous for its art galleries, museums, shopping, and lively nightlife.\n\n3. The Highlands - Explore the stunning landscapes of the Scottish Highlands, including Loch Ness, Glencoe, and the Isle of Skye.\n\n4. Stirling - Visit Stirling Castle, the Wallace Monument, and enjoy the picturesque views of the surrounding countryside.\n\n5. Inverness - Gateway to the Highlands, Inverness is a charming city with a rich history and close proximity to Loch Ness.\n\n6. Aberdeen - Known as th

### Observation:
The RAG-enhanced LLM is using its internal knowledge here. So far, the issue I have with building this chatbot is controlling the chabot's behavior in retrieving information.

## Enabling the conversational application to access the Internet
There can be cases where the internatal and external knowledge of the LLM is not enough for the conversational chatbot to give the most appropriate answer. For example, if I want to ask the conversational chatbot to plan my trip to Japan according to next week's weather, the chatbot needs information concerning next week's weather from the Japan Met Office.

In [21]:
# SerpAPI has a free limit of 100 calls per month
from langchain.utilities import SerpAPIWrapper
import os
from dotenv import load_dotenv
load_dotenv()

serpapi_key = os.getenv("SERPAPI_API_KEY")


In [22]:
search = SerpAPIWrapper(serpapi_api_key=serpapi_key)

from langchain.tools import Tool
from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.agents.agent_toolkits import create_conversational_retrieval_agent

tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    create_retriever_tool(
        db.as_retriever(),
        "japan_travel",
        "Searches and returns documents regarding Japan."
    )
    ]
agent_executor = create_conversational_retrieval_agent(llm, tools, memory_key='chat_history', verbose=True)

In [23]:
while True:
    user_input = input("You: ")
    
    if user_input.lower() in ["exit", "quit", "bye"]:
        print("Goodbye!")
        break

    response = agent_executor.invoke({"input": user_input})
    
    print("AI:", response["output"])



> Entering new AgentExecutor chain...

Invoking: `Search` with `weather forecast Tokyo next week`


{'type': 'weather_result', 'temperature': '59', 'unit': 'Fahrenheit', 'precipitation': '20%', 'location': 'Tokyo, Japan', 'date': 'Friday', 'weather': 'Clear with periodic clouds'}The weather forecast for Tokyo next week indicates a temperature of 59°F with a 20% chance of precipitation. The forecast for Friday shows clear skies with periodic clouds.

> Finished chain.
AI: The weather forecast for Tokyo next week indicates a temperature of 59°F with a 20% chance of precipitation. The forecast for Friday shows clear skies with periodic clouds.
Goodbye!


### Observation:
To successfully incorporate Internet search through SerpAPI, 3 code boxes in this section must be ran one after another. If there is a time gap between each of these code boxes, the conversational application will just have internal and external knowledge. 